<a href="https://colab.research.google.com/github/Meguazy/HACK_4DS/blob/main/notebooks_fra/lgb_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Meguazy/HACK_4DS.git

fatal: destination path 'HACK_4DS' already exists and is not an empty directory.


In [2]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [1]:
%cd /content/HACK_4DS

/content/HACK_4DS


# Initial

In [2]:
!pip install lightgbm imbalanced-learn

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Preprocessing

In [282]:
df_train = pd.read_csv("data2/train_set.csv", sep=";", quotechar="\"", decimal=",")

In [283]:
df_test = pd.read_csv("data2/test_set.csv", sep=";", quotechar="\"", decimal=",")

In [284]:
external_score_ver03_dictionary = {
    "MISSING":None,
    "A":14,
    "B":13,
    "C":12,
    "D":11,
    "E":10,
    "F":9,
    "G":8,
    "H":7,
    "I":6,
    "L":5,
    "M":4,
    "N":3,
    "O":2,
    "P":1
}

df_train = df_train.replace({"external_score_ver03": external_score_ver03_dictionary})
df_test = df_test.replace({"external_score_ver03": external_score_ver03_dictionary})

In [285]:
# Concatenate df_train and df_test
df_combined = pd.concat([df_train, df_test], axis=0)

# List of columns to encode
columns_to_encode = ['province', 'juridical_form', 'industry_sector', 'region', 'geo_area', 'decision_date']

# Instantiate LabelEncoder
label_encoders = {}

# Encode each column iteratively
for column in columns_to_encode:
    # Instantiate LabelEncoder for the current column
    label_encoders[column] = LabelEncoder()

    # Fit label encoder on combined data
    label_encoders[column].fit(df_combined[column])

    # Transform both training and testing data
    df_train[column] = label_encoders[column].transform(df_train[column])
    df_test[column] = label_encoders[column].transform(df_test[column])

In [286]:
columns_to_drop = ['application_ID', 'target', 'late_payment_score', 'external_score_late_payment_integrated', 'external_score_moderate', 'external_score_adverse']

# Drop the columns from the DataFrame
df_train = df_train.drop(columns=columns_to_drop)
df_train_no_target = df_train.drop(columns=['days_to_default'])

df_test = df_test.drop(columns=['application_ID', 'late_payment_score', 'external_score_late_payment_integrated', 'external_score_moderate', 'external_score_adverse'])

# Feature Engineering

In [287]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_train_no_target, df_train.days_to_default, test_size=0.2, random_state=42)

In [288]:
# Standardize Features

# Use StandardScaler to scale the training and validation data
scaler = StandardScaler()
#Fit the StandardScaler to the training data
scaler.fit(X_train)
# transform both the training and validation data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
df_test = scaler.transform(df_test)

In [289]:
X_test

array([[ 0.48641614,  0.7614533 ,  0.69458735, ...,  0.95529891,
         2.57105078,  3.06299297],
       [ 0.54963871,  1.40932541, -0.44050406, ..., -0.47543632,
        -0.1001505 ,  0.01608175],
       [ 0.29147989, -1.13944533,  0.69458735, ..., -0.47543632,
        -0.1001505 ,  0.01608175],
       ...,
       [ 1.46636594, -1.28485297,  0.12704165, ..., -0.47543632,
         0.56764982,  0.77780955],
       [-0.77276667,  0.13254037, -1.57559547, ...,  0.80061152,
         1.90325046,  2.30126517],
       [ 0.91843702,  1.45130423, -1.00804976, ..., -0.47543632,
        -0.76795082, -0.74564606]])

In [290]:
df_train.sort_index()

,decision_date,company_ID,external_score_ver01,external_score_ver02,external_score_ver03,age,province,juridical_form,industry_sector,gross_margin_ratio,...,last_rel_used_a_scadenza,avg_count_enti_affidanti,std_count_enti_affidanti,max_count_enti_affidanti,last_count_enti_affidanti,avg_count_numero_prima_info,std_count_numero_prima_info,max_count_numero_prima_info,last_count_numero_prima_info,days_to_default
0,714,7.256588e+09,10,3,NaN,15,94,13,16,0.464637,...,0.172032,1.000000,0.000000,1,1,0.0,0.000000,0,0,522
1,337,6.178307e+09,7,3,7.0,5,19,8,4,0.372340,...,0.146385,1.000000,0.000000,1,1,2.0,0.000000,2,2,1498
2,155,7.692855e+09,7,3,NaN,5,44,13,5,0.270000,...,0.000000,1.000000,0.000000,1,1,1.0,0.000000,1,1,779
3,762,5.752242e+09,8,2,NaN,6,79,13,16,0.419929,...,0.050167,1.000000,0.000000,1,1,0.5,0.522233,1,0,1498
4,639,7.533507e+09,4,1,NaN,5,63,13,16,0.526316,...,0.000000,0.000000,0.000000,0,0,0.0,0.000000,0,0,1498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32027,464,4.270447e+09,8,1,10.0,7,104,13,6,0.248980,...,0.505934,7.083333,1.311372,8,7,1.5,2.713602,6,0,894
32028,512,7.809509e+09,5,1,5.0,5,47,13,5,-0.006410,...,0.003317,4.583333,0.900337,6,5,4.0,0.000000,4,4,1498
32029,604,3.093873e+09,5,1,10.0,4,10,13,16,0.454158,...,0.318381,3.000000,0.000000,3,3,1.0,0.000000,1,1,1498
32030,900,5.879529e+09,7,1,9.0,6,55,13,7,0.427223,...,0.114085,5.333333,0.492366,6,6,0.0,0.000000,0,0,87


# Hyperparameter tuning

In [ ]:

# Import necessary libraries for calculating mean squared error and using the LightGBM regressor.
from sklearn.metrics import mean_absolute_error as mae
from lightgbm import LGBMRegressor
start = 0
end = 100
step = 1

# Initialize an empty list to store the values
values = []

# Loop with decimal step
value = start
while value <= end:
    value += step
    model = LGBMRegressor(
        objective='regression',
        boosting_type='gbdt',
        num_leaves=51,
        learning_rate=0.18,
        n_estimators=200,
        feature_fraction=0.88,
        bagging_fraction=0.8,
        bagging_freq=value,
        verbose=-1,
    )
    model.fit(X_train, y_train)

    # Make predictions on the training and validation data.
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)

    # Calculate and print the Root Mean Absolute Error (MAE) for training and validation predictions.

    print(value)
    print("Training MAE: ", mae(y_train, y_train_pred))
    print("Validation MAE: ", mae(y_test, y_pred))


    values.append((value, mae(y_test, y_pred)))

# MODEL TRAINING

In [291]:
value += step
model = LGBMRegressor(
    objective='regression',
    boosting_type='gbdt',
    num_leaves=51,
    learning_rate=0.18,
    n_estimators=200,
    feature_fraction=0.88,
    bagging_fraction=0.8,
    num_iterations=5000,
    bagging_freq=5,
    verbose=-1,
)
model.fit(X_train, y_train)

# Make predictions on the training and validation data.
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

# Calculate and print the Root Mean Absolute Error (MAE) for training and validation predictions.
print("Training MAE: ", mae(y_train, y_train_pred))
print("Validation MAE: ", mae(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training MAE:  0.03201633718901852
Validation MAE:  246.49853667410216


# Submit solution

In [294]:
# Making predictions
y_test_pred = model.predict(df_test)

In [295]:
submission = pd.DataFrame(y_test_pred, columns=["label"])

In [296]:
submission.to_csv("file.csv", index=False, header=False)